In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
import pickle
from actsnclass.metrics import get_snpcc_metric
from shutil import copyfile
from sklearn.metrics import roc_curve, auc
from scipy.interpolate import interp1d

In [1]:
import sklearn

In [2]:
sklearn.__version__

'1.0.2'

In [2]:
dirname = '../../../referee/'

In [3]:
all_fom = []

for i in range(100):
    fname = dirname + '/UncSampling/metrics/metrics_UncSampling_v' + str(i) + '.dat'
    data = pd.read_csv(fname, index_col=False, delim_whitespace=True)
    all_fom.append([i, data.iloc[299]['fom']])
    
all_fom = np.array(all_fom)

In [4]:
fom_max = np.max(all_fom[:,1])

In [5]:
distance =  abs(all_fom[ :,1] - fom_max)
indx = np.argsort(distance)

In [6]:
distance[indx[0]]

0.0

In [7]:
indx[0]

68

In [8]:
all_fom[indx[0]]

array([68.        ,  0.42013397])

In [9]:
fname = dirname + '/UncSampling/training_samples/initialtrain_v' + str(indx[0]) + '.csv'
data_train = pd.read_csv(fname, index_col=False)

fname = dirname + '/UncSampling/test_samples/initial_test_v' + str(indx[0]) + '.csv'
data_test = pd.read_csv(fname, index_col=False)

fname = dirname + '/UncSampling/queries/queried_UncSampling_v' + str(indx[0]) + '.dat'
data_query = pd.read_csv(fname, index_col=False, delim_whitespace=True)

In [10]:
flag = np.array([item in data_query['id'].values for item in data_test['objectId'].values])

data_train2 = pd.concat([data_train, data_test[flag]], ignore_index=True)
data_test2 = data_test[~flag]

In [11]:
clf = RandomForestClassifier(random_state=42, n_estimators=1000)
clf.fit(data_train2[list(data_train2.keys())[:-2]], data_train2['type'].values == 'Ia')

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [12]:
clf.score(data_test2[list(data_test2.keys())[:-2]], data_test2['type'].values == 'Ia')

0.967824419347965

In [13]:
pred = clf.predict(data_test2[list(data_test2.keys())[:-2]])

In [14]:
get_snpcc_metric(pred, data_test2['type'].values == 'Ia')

(['accuracy', 'efficiency', 'purity', 'fom'],
 [0.967824419347965,
  0.5372522214627478,
  0.9097222222222222,
  0.4140002412448233])

In [18]:
name_prob = dirname + '/UncSampling/class_prob/v' + str(i) + \
                 '/class_prob_UncSampling_loop_299.csv'
data_prob = pd.read_csv(name_prob, index_col=False)
    
label = data_prob['type'].values == 'Ia'
fpr, tpr, thresholds = roc_curve(label, data_prob['prob_Ia'].values)
roc_auc = auc(fpr, tpr)
    
fpr_int = interp1d(thresholds, fpr)
tpr_int = interp1d(thresholds, tpr)

# Identify alerts for which we should NOT provide classification

In [21]:
fname_train = dirname + 'UncSampling/training_samples/initialtrain_v' + str(indx[0]) + '.csv'
data_train = pd.read_csv(fname_train)

fname_query = dirname + 'UncSampling/queries/queried_UncSampling_v' + str(indx[0]) + '.dat'
data_query = pd.read_csv(fname_query, delim_whitespace=True, index_col=False)

In [22]:
all_ids = list(data_train['objectId'].values) + list(data_query['id'].values)